In [10]:
from src_clean.dataloader.dataloader import DataLoader
from src_clean.preprocessor.prepro_new import PreprocessorTwo

In [11]:
train_df, test_df = DataLoader.load_train_test_dfs()
#train_df = PreprocessorTwo.preprocess(train_df, exclude_cols_to_drop=['srch_id', 'prop_id', 'position'])
train_df = PreprocessorTwo.remove_outliers(train_df)
train_df = PreprocessorTwo.relevance(train_df)
train_df = PreprocessorTwo.prepro_comp_columns(train_df)

#test_df = PreprocessorTwo.preprocess(test_df, exclude_cols_to_drop=['prop_id'])
test_df = PreprocessorTwo.prepro_comp_columns(test_df)
train_df, test_df = PreprocessorTwo.calculate_statistics(train_df, test_df)
train_df, test_df = PreprocessorTwo.mean_pos_per_prop_id(train_df, test_df)
#train_df, test_df = PreprocessorTwo.click_and_booking_prob(train_df, test_df)

train_df = PreprocessorTwo.drop_cols(train_df, exclude_cols_to_drop=['srch_id', 'prop_id'])
test_df = PreprocessorTwo.drop_cols(test_df)

In [12]:
train_df, test_df = PreprocessorTwo.impute(train_df, test_df)

In [13]:
X_train, _, y_train, _ = DataLoader.split_df_into_train_and_val_batches(train_df, validation_size=0)

In [5]:
len(X_train)

199734

In [14]:
X_train = X_train[:round(len(X_train)/10)]
y_train = y_train[:round(len(y_train)/10)]

In [7]:
len(X_train)

3995

In [24]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
import numpy as np

regr = LinearSVR(par)
regr.fit(np.vstack(X_train), np.hstack(y_train))

LinearSVR()

In [25]:
from src_clean.score.ndcg import calculate_ndcg
from src_clean.ranker.ranker import Ranker

pred = regr.predict(test_df.to_numpy())
ranker = Ranker()
ranking_file_path = ranker.make_ranking_from_prediction(
    pred, model_name="LightGBM"
)
calculate_ndcg(ranking_file_path)

0.242904958157556

In [17]:
pred

array([0.33805327, 0.21060219, 0.32913991, ..., 0.44602097, 0.31330397,
       0.37795051])